In [7]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf


import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns


import requests
import csv
import json

import warnings

In [8]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")
NASDAQ_DATA_LINK_API_KEY = os.getenv("NASDAQ_DATA_LINK_API_KEY")

In [3]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [4]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

ticker_symbols = [

#     # energy
    'SLB'
#     # ,'HAL'
#     # ,'OKE'

#     # consumer staples
    ,'TGT'
#     # ,'STZ'
#     # ,'DLTR'
#     # ,'DG'
#     # ,'TSN'
    ,'MNST'
#     # ,'PG'
#     # ,'PM'
#     # ,'MO'


#     # finanace
    ,'TRV'
#     # ,'CB'
#     # ,'BAC'

#     # technology
#     # ,'NVDA'
#     ,'MSFT'

#     # INDEX
#     # ,'SPY'
#     # ,'QQQ'

    
#     # holding stock
    ,'TSM'
#     ,'ACN'
#     ,'DHI'
#     # ,'STZ'
    ,'GOOG'
#     # ,'TGT'
]

# ticker_symbols = [
#     'MO',
#     'ADM',
#     # 'BF.B', # DATA ISSUE
#     'BG',
#     'CPB',
#     'CHD',
#     'CLX',
#     'KO',
#     'CL',
#     'CAG',
#     'STZ', # !!
#     'COST',
#     'DG',
#     'DLTR', # !!
#     'EL',
#     'GIS',
#     'HSY',
#     'HRL',
#     'K',
#     'KVUE',
#     'KDP',
#     'KMB',
#     'KHC',
#     # 'KR', # DATA ISSUE
#     'LW',
#     # 'MKC', # DATA ISSUE
#     'TAP',
#     'MDLZ',
#     'MNST',
#     'PEP',
#     'PM',
#     'PG',
#     'SJM',
#     'SYY',
#     'TGT', # !!
#     'TSN', # !!
#     'WBA', # DATA ISSUE
#     'WMT'
# ]

# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 6 # this will return x-1 yr PE range

ticker_dict_json = {}
ticker_dict_pd = {}

# PE TTM Valuation

In [5]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)


    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - 6) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue

            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values
            annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df["PE"].mean().round(2)
            annualEPS_df[f"PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df["PE"]).round(2)
            annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
            annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API
    # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        forecast_earanings_df = pd.DataFrame(
            columns=my_list[0]
            ,data=my_list[1::]
            )
        
        if forecast_earanings_df['estimate'].head(1).values != '':
            latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
        else:
            latest_projected_EPS = 0


    # forecast 1 year earnings from yf API
    yf_data = yf.Ticker(symbol).info

    forecast_earnings_keys = [
        'forwardEps'
        ,'forwardPE'
        ,'pegRatio'
    ]


    if not all(key in yf_data.keys() for key in forecast_earnings_keys):
        # Handle the case where one or more keys are missing
        EPS_12month_projected = 0
        PE_12month_projected = 0
        PEG_12month_projected = 0
    else:
        EPS_12month_projected = yf_data['forwardEps'] # 代表了截止下一个日历年结束的EPS, next year forecasted EPS
        PE_12month_projected = yf_data['forwardPE'] # 代表了截止下一个日历年结束的PE, next year forecasted PE
        PEG_12month_projected = yf_data['pegRatio'] # 代表了截止下一个日历年结束的PEG 


    # US Treasury section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            US_T_10yrs_df = pd.DataFrame(value)
            US_T_10yrs_df['value'] = pd.to_numeric(US_T_10yrs_df['value'], errors='coerce') # change dataframe value to numeric data
            US_T_10yrs_YTM = US_T_10yrs_df['value'][0]




    # Consolidated section
    stock_consolidate_df = Daily_stock_df.head(window_days)


    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM

        if i == max(stock_consolidate_df.index):
            EPS_latest_projected = latest_projected_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        stock_consolidate_df["EPS_latest_projected"] = EPS_latest_projected
    

    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    stock_consolidate_df["PE_1yr_forward"] = round(PE_12month_projected, 2)
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    stock_consolidate_df["relative_valuation_nextQuater_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_latest_projected"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextQuater_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_latest_projected"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextQuater_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextQuater_projected_+"][0], stock_consolidate_df["relative_valuation_nextQuater_projected_-"][0]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    stock_consolidate_df[f"{window_days}_price_min"] = stock_consolidate_df["stock_price"].min().round(2)
    stock_consolidate_df[f"{window_days}_price_max"] = stock_consolidate_df["stock_price"].max().round(2)
    stock_consolidate_df[f"{window_days}_price_avg"] = stock_consolidate_df["stock_price"].mean().round(2)
    stock_consolidate_df[f"{window_days}_price_std"] = np.std(stock_consolidate_df["stock_price"]).round(2)

    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_std"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    stock_consolidate_df["relative_valuation_nextYear_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextYear_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextYear_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextYear_projected_+"][0], stock_consolidate_df["relative_valuation_nextYear_projected_-"][0]])).round(2) #这个是根据next year projected EPS 得出PE估值的价格中位数

    stock_consolidate_df["next12months_PEG"] = PEG_12month_projected
    stock_consolidate_df["TTM_PEG"] = (stock_consolidate_df["PE_TTM"] / (((EPS_12month_projected - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100)).round(2) # 这个是截止下一年的EPS growth rate所得出的PEG ratio, <1是undervalue的表现
    
    stock_consolidate_df["nextYear_EPS_growthRate"] = (((EPS_12month_projected - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)
    stock_consolidate_df["nextQuater_EPS_growthRate"] = (((stock_consolidate_df["EPS_latest_projected"] - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)

    stock_consolidate_df["EarningYield_TTM"] = ((stock_consolidate_df["EPS_TTM"] / stock_consolidate_df["stock_price"]) * 100).round(2)
    stock_consolidate_df["ERP_TTM"] = stock_consolidate_df["EarningYield_TTM"] - US_T_10yrs_YTM




    # filter conditions
    conditions = [
    (stock_consolidate_df["stock_price"] < stock_consolidate_df["relative_valuation_TTM_-"]),
    (stock_consolidate_df["stock_price"] > stock_consolidate_df["relative_valuation_TTM_+"]),
    ((stock_consolidate_df["stock_price"] >= stock_consolidate_df["relative_valuation_TTM_-"]) & (stock_consolidate_df["stock_price"] <= stock_consolidate_df["relative_valuation_TTM_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    stock_consolidate_df["curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        stock_consolidate_df.loc[condition, "price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    "Ticker"
    ,"stock_price"
    ,"EPS_TTM"
    ,"EPS_latest_projected"
    ,"PE_1yr_forward"
    ,"PE_TTM"
    ,"PE_TTM_avg"
    ,f"PE_{PE_yr_range-1}yr_avg"
    ,f"PE_{PE_yr_range-1}yr_volatility_+"
    ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,"relative_valuation_nextQuater_projected_+"
    ,"relative_valuation_nextQuater_projected_-"
    ,"relative_valuation_nextQuater_projected_median"
    ,"relative_valuation_nextYear_projected_+"
    ,"relative_valuation_nextYear_projected_-"
    ,"relative_valuation_nextYear_projected_median"
    ,"price_valuation_assessment"
    ,"nextQuater_EPS_growthRate"
    ,"nextYear_EPS_growthRate"
    ,"next12months_PEG"
    ,"TTM_PEG"
    ,"EarningYield_TTM"
    ,"ERP_TTM"
    ]


    # store each stock info as pd into dictionary
    ticker_dict_pd[symbol] = stock_consolidate_df[selected_cols]
    # transfer pandas dataframe to json format, and each stock info into dictionary
    ticker_dict_json[symbol] = stock_consolidate_df[selected_cols].to_json()

    if j == 0:
        # screener df creation
        # screener df will store each stock's consolidate df's first row and union them together for screening purposee
        ticker_screen_df = pd.DataFrame(
            columns=selected_cols
        ) 
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]

        # Insert rows into the DataFrame
        ticker_screen_df.loc[j] = stock_consolidate_df_values

    else:
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]
        ticker_screen_df.loc[j] = stock_consolidate_df_values

In [6]:
ticker_dict_pd['MNST']

,Ticker,stock_price,EPS_TTM,EPS_latest_projected,PE_1yr_forward,PE_TTM,PE_TTM_avg,PE_5yr_avg,PE_5yr_volatility_+,PE_5yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,nextQuater_EPS_growthRate,nextYear_EPS_growthRate,next12months_PEG,TTM_PEG,EarningYield_TTM,ERP_TTM
2024-08-08,MNST,45.00,1.64,1.66,23.22,27.44,32.37,35.32,41.03,29.61,55.83,50.35,53.09,56.51,50.96,53.74,67.06,60.48,63.77,undervalued,1.22,20.12,1.87,1.36,3.64,-0.32
2024-08-07,MNST,50.53,1.62,1.66,23.22,31.19,32.37,35.32,41.03,29.61,55.14,49.73,53.09,56.51,50.96,53.74,67.06,60.48,63.77,fair,2.47,21.60,1.87,1.44,3.21,-0.75
2024-08-06,MNST,50.80,1.62,1.66,23.22,31.36,32.37,35.32,41.03,29.61,55.14,49.73,53.09,56.51,50.96,53.74,67.06,60.48,63.77,fair,2.47,21.60,1.87,1.45,3.19,-0.77
2024-08-05,MNST,50.47,1.62,1.66,23.22,31.15,32.37,35.32,41.03,29.61,55.14,49.73,53.09,56.51,50.96,53.74,67.06,60.48,63.77,fair,2.47,21.60,1.87,1.44,3.21,-0.75
2024-08-02,MNST,51.61,1.62,1.66,23.22,31.86,32.37,35.32,41.03,29.61,55.14,49.73,53.09,56.51,50.96,53.74,67.06,60.48,63.77,fair,2.47,21.60,1.87,1.47,3.14,-0.82
2024-08-01,MNST,51.38,1.62,1.66,23.22,31.72,32.37,35.32,41.03,29.61,55.14,49.73,53.09,56.51,50.96,53.74,67.06,60.48,63.77,fair,2.47,21.60,1.87,1.47,3.15,-0.81
2024-07-31,MNST,51.45,1.62,1.66,23.22,31.76,32.37,35.32,41.03,29.61,55.14,49.73,53.09,56.51,50.96,53.74,67.06,60.48,63.77,fair,2.47,21.60,1.87,1.47,3.15,-0.81
2024-07-30,MNST,51.57,1.62,1.66,23.22,31.83,32.37,35.32,41.03,29.61,55.14,49.73,53.09,56.51,50.96,53.74,67.06,60.48,63.77,fair,2.47,21.60,1.87,1.47,3.14,-0.82
2024-07-29,MNST,51.27,1.62,1.66,23.22,31.65,32.37,35.32,41.03,29.61,55.14,49.73,53.09,56.51,50.96,53.74,67.06,60.48,63.77,fair,2.47,21.60,1.87,1.46,3.16,-0.80
2024-07-26,MNST,51.00,1.62,1.66,23.22,31.48,32.37,35.32,41.03,29.61,55.14,49.73,53.09,56.51,50.96,53.74,67.06,60.48,63.77,fair,2.47,21.60,1.87,1.46,3.18,-0.78


In [150]:
ticker_screen_df.sort_values(by=['ERP_TTM']
                             ,ascending=False)

,Ticker,stock_price,EPS_TTM,EPS_latest_projected,PE_1yr_forward,PE_TTM,PE_TTM_avg,PE_5yr_avg,PE_5yr_volatility_+,PE_5yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,nextQuater_EPS_growthRate,nextYear_EPS_growthRate,next12months_PEG,TTM_PEG,EarningYield_TTM,ERP_TTM
33,WBA,11.87,3.16,2.85,6.35,3.76,4.59,8.99,10.67,7.31,16.72,12.29,14.5,15.08,11.09,13.08,10.74,7.9,9.32,undervalued,-9.81,-35.76,-0.31,-0.11,26.62,22.47
2,BG,105.23,13.45,9.73,10.7,7.82,7.86,10.98,16.12,5.84,109.48,101.95,105.72,79.2,73.75,76.47,80.02,74.51,77.26,fair,-27.66,-26.91,-1.81,-0.29,12.78,8.63
23,TAP,52.85,5.84,5.74,9.13,9.05,10.01,11.59,12.09,11.09,66.75,50.17,58.46,65.61,49.31,57.46,66.18,49.74,57.96,fair,-1.71,-0.86,2.85,-10.57,11.05,6.9
0,MO,49.01,4.92,3.61,10.02,9.96,9.18,10.25,11.72,8.78,47.67,42.66,45.16,34.98,31.3,33.14,51.16,45.78,48.47,overvalued,-26.63,7.32,3.1,1.36,10.04,5.89
1,ADM,62.01,5.48,5.25,11.44,11.32,9.51,13.09,15.39,10.79,54.8,49.43,52.11,52.5,47.36,49.93,54.2,48.89,51.54,overvalued,-4.2,-1.09,-2.71,-10.34,8.84,4.69
8,CAG,30.32,2.67,2.6,11.23,11.36,11.1,13.78,16.6,10.96,30.62,28.65,29.64,29.82,27.9,28.86,30.97,28.97,29.97,fair,-2.62,1.12,5.85,10.11,8.81,4.66
21,KHC,35.21,2.98,2.19,12.02,11.82,11.75,12.48,13.52,11.44,37.07,32.96,35.02,27.24,24.22,25.73,39.43,35.06,37.25,fair,-26.51,6.38,3.77,1.85,8.46,4.31
29,SJM,117.95,9.94,9.9,10.85,11.87,11.5,14.05,16.05,12.05,118.78,109.84,114.31,118.3,109.4,113.85,129.9,120.11,125.0,fair,-0.4,9.36,2.12,1.27,8.43,4.28
22,LW,60.02,5.06,4.15,11.72,11.86,15.15,30.19,36.45,23.93,84.2,69.12,76.66,69.06,56.69,62.88,85.2,69.94,77.57,undervalued,-17.98,1.19,0.91,10.0,8.43,4.28
17,K,58.15,4.07,3.01,66.84,14.29,14.25,15.33,16.95,13.71,60.28,55.72,58.0,44.58,41.21,42.9,12.88,11.91,12.4,fair,-26.04,-78.62,-0.47,-0.18,7.0,2.85


In [37]:
# ticker_dict_pd['TSM']

In [21]:
import requests
ticker_symbols = ['MSFT']
for symbol in ticker_symbols:
    print(symbol)

    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    
    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        print(key, value)

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')

            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])


    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]

    # calculating new ratios
    stock_ratios_annual_consolidate_df['ROE'] = annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']

    

MSFT
symbol MSFT
annualReports [{'fiscalDateEnding': '2023-06-30', 'reportedCurrency': 'USD', 'totalAssets': '411976000000', 'totalCurrentAssets': '184257000000', 'cashAndCashEquivalentsAtCarryingValue': '34704000000', 'cashAndShortTermInvestments': '111262000000', 'inventory': '2500000000', 'currentNetReceivables': '48688000000', 'totalNonCurrentAssets': '232219000000', 'propertyPlantEquipment': '95641000000', 'accumulatedDepreciationAmortizationPPE': '68251000000', 'intangibleAssets': '77252000000', 'intangibleAssetsExcludingGoodwill': '9366000000', 'goodwill': '67886000000', 'investments': '95599000000', 'longTermInvestments': '9879000000', 'shortTermInvestments': '76558000000', 'otherCurrentAssets': '21807000000', 'otherNonCurrentAssets': '30601000000', 'totalLiabilities': '205753000000', 'totalCurrentLiabilities': '104149000000', 'currentAccountsPayable': '18095000000', 'deferredRevenue': '107626000000', 'currentDebt': '5247000000', 'shortTermDebt': '0', 'totalNonCurrentLiabilitie

In [19]:
stock_ratios_annual_consolidate_df

,gross_margin,operating_margin,net_margin,netIncome,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,totalShareholderEquity,ROE
13,80.162922,38.566673,30.023686,18760000000,212.934562,29529000000,16.593857,86.492691,210.104410,46175000000,0.406280
12,77.729008,38.833050,33.098380,23150000000,260.366998,46144000000,25.788835,90.431477,255.598804,57083000000,0.405550
11,76.221803,29.519960,23.029448,16978000000,260.291238,52396000000,22.711657,82.738876,256.812898,66363000000,0.255835
10,73.814693,34.379375,28.083855,21863000000,271.176203,64049000000,24.356352,80.420298,265.996739,78944000000,0.276943
9,68.816003,31.968261,25.421211,22074000000,250.402192,68621000000,30.019965,91.998574,244.572055,89784000000,0.245857
8,64.695448,19.406925,13.029493,12193000000,247.340222,73150000000,41.421736,117.863966,241.494954,80083000000,0.152255
7,64.038879,28.608728,22.532198,20539000000,235.288172,80303000000,50.558510,146.192081,231.495864,83090000000,0.247190
6,64.522476,30.055607,26.394052,25489000000,291.857566,106951000000,72.089088,185.382677,287.945107,87711000000,0.290602
5,67.156927,32.696648,15.454851,16571000000,290.080016,111174000000,68.541206,212.928262,285.528655,82718000000,0.200331
4,67.154946,34.786024,31.774566,39240000000,252.883895,106132000000,68.007764,180.031271,249.912129,102330000000,0.383465


In [175]:
# 90% accurate
annual_income_df[income_ratio_cols + ['fiscalDateEnding']]  

,gross_margin,operating_margin,net_margin,fiscalDateEnding
13,80.162922,38.566673,30.023686,2010-06-30
12,77.729008,38.833050,33.098380,2011-06-30
11,76.221803,29.519960,23.029448,2012-06-30
10,73.814693,34.379375,28.083855,2013-06-30
9,68.816003,31.968261,25.421211,2014-06-30
8,64.695448,19.406925,13.029493,2015-06-30
7,64.038879,28.608728,22.532198,2016-06-30
6,64.522476,30.055607,26.394052,2017-06-30
5,67.156927,32.696648,15.454851,2018-06-30
4,67.154946,34.786024,31.774566,2019-06-30


In [168]:
# not accurate, cannot be used
annual_balancesheet_df[balancesheet_ratio_cols + ['fiscalDateEnding']]

,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,fiscalDateEnding
15,0.001278,-78250000000,-7.588498,333.002883,NaN,2008-12-31
14,976.481836,4584000000,134.249564,299.635236,1026.959847,2009-12-31
13,1555.080214,5442000000,134.490996,314.743867,NaN,2010-12-31
12,749.427481,3403000000,236.849838,327.237815,NaN,2011-12-31
11,318.015013,2614000000,553.634277,313.060421,NaN,2012-12-31
10,1321.518987,3860000000,162.202073,318.664301,NaN,2013-12-31
9,506.737968,3803000000,69.681830,315.034627,NaN,2014-12-31
8,616.727717,4232000000,55.529301,324.544453,NaN,2015-12-31
7,569.603524,4264000000,45.731707,331.699755,NaN,2016-12-31
6,586.725664,4400000000,32.295455,336.066748,NaN,2017-12-31


In [176]:
# # List of columns to plot
# # columns_to_plot = qtr_income_df.columns[1:]  # Exclude the 'fiscalDateEnding' column

# # Create bar charts for each column
# for column in qtr_income_ratio_cols:
#     fig = px.bar(qtr_income_df, x='fiscalDateEnding', y=column, title=column)
#     fig.show()

In [11]:
symbol = 'MSFT'

# BALANCESHEET
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
r = requests.get(url)
data = r.json()

for key, value in data.items():
    print(key, value)

    if key == 'annualReports':
        annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
        annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

        for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
            annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')

    if key == 'quarterlyReports':
        qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
        qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

        for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
            qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')

symbol MSFT
annualReports [{'fiscalDateEnding': '2023-06-30', 'reportedCurrency': 'USD', 'totalAssets': '411976000000', 'totalCurrentAssets': '184257000000', 'cashAndCashEquivalentsAtCarryingValue': '34704000000', 'cashAndShortTermInvestments': '111262000000', 'inventory': '2500000000', 'currentNetReceivables': '48688000000', 'totalNonCurrentAssets': '232219000000', 'propertyPlantEquipment': '95641000000', 'accumulatedDepreciationAmortizationPPE': '68251000000', 'intangibleAssets': '77252000000', 'intangibleAssetsExcludingGoodwill': '9366000000', 'goodwill': '67886000000', 'investments': '95599000000', 'longTermInvestments': '9879000000', 'shortTermInvestments': '76558000000', 'otherCurrentAssets': '21807000000', 'otherNonCurrentAssets': '30601000000', 'totalLiabilities': '205753000000', 'totalCurrentLiabilities': '104149000000', 'currentAccountsPayable': '18095000000', 'deferredRevenue': '107626000000', 'currentDebt': '5247000000', 'shortTermDebt': '0', 'totalNonCurrentLiabilities': '

In [14]:
qtr_balancesheet_df.tail(5)

,fiscalDateEnding,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
4,2023-03-31,380088000000,163889000000,26562000000,104427000000,2877000000,38137000000,220499000000,88132000000,65998000000,77819000000,9879000000,67940000000,9.518600e+10,9415000000,77865000000,19165000000,26954000000,185405000000,85691000000,15305000000,7.920200e+10,6.245000e+09,6245000000,99714000000,1.231200e+10,48210000000,6.245000e+09,41965000000,48210000000,12664000000,17437000000,194683000000,NaN,108234000000,92093000000,7437000000
3,2023-06-30,411976000000,184257000000,34704000000,111262000000,2500000000,48688000000,232219000000,95641000000,68251000000,77252000000,9366000000,67886000000,9.559900e+10,9879000000,76558000000,21807000000,30601000000,205753000000,104149000000,18095000000,1.076260e+11,5.247000e+09,0,101604000000,1.272800e+10,47237000000,5.247000e+09,41990000000,47237000000,14745000000,17981000000,206223000000,NaN,118848000000,93718000000,7432000000
2,2023-09-30,445785000000,207586000000,80452000000,143951000000,3000000000,36953000000,242699000000,102502000000,69486000000,76685000000,8895000000,67790000000,6.875000e+10,11423000000,63499000000,23682000000,32154000000,225071000000,124792000000,19307000000,9.837600e+10,2.955600e+10,25808000000,100279000000,1.348700e+10,45694000000,3.748000e+09,41946000000,71502000000,14475000000,18634000000,220714000000,NaN,132143000000,95508000000,7431000000
1,2023-12-31,470558000000,147393000000,17305000000,81017000000,1615000000,42831000000,327965000000,112308000000,72949000000,148827000000,29896000000,118931000000,7.148200e+10,13367000000,63712000000,21930000000,32265000000,232290000000,121016000000,17695000000,9.206800e+10,2.929100e+10,27041000000,111274000000,1.415500e+10,47178000000,2.250000e+09,44928000000,74219000000,16362000000,20787000000,238268000000,NaN,145737000000,97480000000,7432000000
0,2024-03-31,484275000000,147180000000,19634000000,80021000000,1304000000,44029000000,341895000000,121375000000,74945000000,147991000000,28828000000,119163000000,6.979100e+10,14807000000,60387000000,21826000000,35551000000,231123000000,118525000000,18087000000,8.966600e+10,2.274900e+10,20500000000,112598000000,1.446900e+10,44907000000,2.249000e+09,42658000000,65407000000,18023000000,23271000000,253152000000,NaN,159394000000,99193000000,7433000000


# SP500 sectors

In [38]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Consumer Staples']['Ticker'].tolist()

In [39]:
# Pivot the DataFrame
pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
pivot_sector_ticker_df = pivot_sector_ticker_df.set_index('Sector').T

pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBWI, BBY, BKNG, BWA, ...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, ACGL, AJG, AIZ,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, BIO, TECH...","[MMM, AOS, ALLE, AAL, AME, ADP, AXON, BA, BR, ...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CE, CF, CTVA, DOW,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [15]:
pivot_sector_ticker_df['Financials'].iloc[0]

['AFL',
 'ALL',
 'AXP',
 'AIG',
 'AMP',
 'AON',
 'ACGL',
 'AJG',
 'AIZ',
 'BAC',
 'BK',
 'BRK.B',
 'BLK',
 'BX',
 'BRO',
 'COF',
 'CBOE',
 'SCHW',
 'CB',
 'CINF',
 'C',
 'CFG',
 'CME',
 'CPAY',
 'DFS',
 'EG',
 'FDS',
 'FIS',
 'FITB',
 'FI',
 'BEN',
 'GPN',
 'GL',
 'GS',
 'HIG',
 'HBAN',
 'ICE',
 'IVZ',
 'JKHY',
 'JPM',
 'KEY',
 'KKR',
 'L',
 'MTB',
 'MKTX',
 'MMC',
 'MA',
 'MET',
 'MCO',
 'MS',
 'MSCI',
 'NDAQ',
 'NTRS',
 'PYPL',
 'PNC',
 'PFG',
 'PGR',
 'PRU',
 'RJF',
 'RF',
 'SPGI',
 'STT',
 'SYF',
 'TROW',
 'TRV',
 'TFC',
 'USB',
 'V',
 'WRB',
 'WFC',
 'WTW']

In [16]:
len(pivot_sector_ticker_df['Financials'].iloc[0])

71

## Appendix

In [128]:
# Consumer Staples

ticker_symbols = [
    'MO',
    'ADM',
    # 'BF.B', # DATA ISSUE
    'BG',
    'CPB',
    'CHD',
    'CLX',
    'KO',
    'CL',
    'CAG',
    'STZ', # !!
    'COST',
    'DG',
    'DLTR', # !!
    'EL',
    'GIS',
    'HSY',
    'HRL',
    'K',
    'KVUE',
    'KDP',
    'KMB',
    'KHC',
    # 'KR', # DATA ISSUE
    'LW',
    # 'MKC', # DATA ISSUE
    'TAP',
    'MDLZ',
    'MNST',
    'PEP',
    'PM',
    'PG',
    'SJM',
    'SYY',
    'TGT', # !!
    'TSN', # !!
    # 'WBA', # DATA ISSUE
    'WMT'
]

In [ ]:
# Energy

ticker_symbols = [
'APA',
 'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
 'CVX',
 'COP',
 'CTRA',
 'DVN',
 'FANG',
 'EOG',
 'EQT',
 'XOM',
 'HAL', # !!!! 与SLB类似
 'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
 'KMI',
 'MRO',
 'MPC',
 'OXY',
 'OKE', # !!
 'PSX',
 'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
 'TRGP', # !！ HGIH PE, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
 'VLO',
 'WMB'
 ]